In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.interpretable_functions as interpret
import utils.RiskSLIM as slim
import utils.stumps as stumps
import utils.fairness_functions as fairness
from utils.load_settings import load_settings

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model

# restore saved variables
%store -r summary_drug2_FL_interpret

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### CART & EBM

In [2]:
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_data.csv").sort_values('person_id')
x = data.loc[:,:'five_year']
y = data['drug_two_year'].values

In [3]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003,0.004,0.005]
cart_summary = interpret.CART(X=x,
                         Y=y,
                         depth=depth,
                         impurity=impurity, 
                         seed = 816)

#### GAM
estimators = [40,60,80,100]
depth = [1,2,3]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(X=x,
                       Y=y,
                       learning_rate = learning_rate,
                       depth = depth,
                       estimators=estimators,
                       holdout_split=holdout_split,
                       seed=816)

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha

In [4]:
np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])

(0.6538781478216074, 0.08366322579357326)

In [5]:
np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])

(0.6808643542744772, 0.10732290145669496)

## Lasso Stumps

In [6]:
## load stumps & original data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_data.csv").sort_values('person_id')
original_data = original_data.loc[:, ['person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
data = pd.merge(original_data, data, on=['person_id', 'screening_date'])

## split X & Y
X_stumps = data.loc[:,:'five_year1'].copy()
Y_stumps = data['drug_two_year'].values.copy()
Y_stumps[Y_stumps == -1] = 0
cols = X_stumps.columns[5:]

## load train & test stumps data
train_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_train_stumps.csv")
test_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_test_stumps.csv")
X_train_stumps = train_stumps.loc[:,:'five_year1'].copy()
X_test_stumps = test_stumps.loc[:,:'five_year1'].copy()
Y_train_stumps = train_stumps['drug_two_year'].values.copy()
Y_test_stumps = test_stumps['drug_two_year'].values.copy()
Y_train_stumps[Y_train_stumps == -1] = 0
Y_test_stumps[Y_test_stumps == -1] = 0

##### Single Stump Model

In [7]:
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.08, 
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

29

In [8]:
single_stump_model['features']

['age_at_current_charge23',
 'age_at_current_charge27',
 'age_at_current_charge29',
 'age_at_current_charge31',
 'age_at_current_charge33',
 'age_at_current_charge57',
 'age_at_first_charge19',
 'age_at_first_charge21',
 'p_charges8',
 'p_probation2',
 'p_probation3',
 'p_felassault1',
 'p_weapon1',
 'p_fta_two_year1',
 'p_pending_charge2',
 'p_pending_charge6',
 'p_misdemeanor1',
 'p_misdemeanor4',
 'p_violence1',
 'p_violence5',
 'total_convictions2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest6',
 'p_drug1',
 'p_drug2',
 'p_drug4',
 'p_dui1',
 'one_year1']

#### Nested Cross Validation

In [9]:
stump_summary = stumps.stump_cv(X = X_stumps, 
                                Y = Y_stumps, 
                                columns=cols, 
                                c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]},
                                seed = 816)

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [10]:
stump_summary['best_params'], np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])

([{'C': 0.08}, {'C': 0.07}, {'C': 0.08}, {'C': 0.08}, {'C': 0.08}],
 0.7176427384116442,
 0.07189410899365092)

##### Best Stump Model

In [11]:
best_stump_model = stumps.stump_model(X_train_stumps, 
                                      Y_train_stumps, 
                                      X_test_stumps, 
                                      Y_test_stumps, 
                                      c=0.08, 
                                      columns=cols, 
                                      seed=816)
best_stump_model['test_auc']

0.7099419334588827

## RiskSLIM -- Unconstrained

In [57]:
## load stumps data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_stumps.csv").sort_values('person_id')

## single stump model
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.05,  
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

16

In [58]:
selected_features = single_stump_model['features']

In [59]:
### Subset features
if 'sex1' in selected_features:
    selected_features = ['drug_two_year', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['drug_two_year', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0

sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on=['person_id', 'screening_date'])
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [60]:
riskslim_summary = slim.risk_nested_cv(X=sub_X, 
                                       Y=sub_Y, 
                                       indicator = indicator,
                                       y_label='drug_two_year', 
                                       max_coef=5,
                                       max_coef_number=5, 
                                       max_runtime=1000, 
                                       c=1e-6, 
                                       max_offset=100,
                                       seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/20/19 @ 11:39 PM | 213 rows in lookup table
11/20/19 @ 11:39 PM | ------------------------------------------------------------
11/20/19 @ 11:39 PM | runnning initialization procedure
11/20/19 @ 11:39 PM | ------------------------------------------------------------
11/20/19 @ 11:39 PM | CPA produced 2 cuts
11/20/19 @ 11:39 PM | running naive rounding on 7 solutions
11/20/19 @ 11:39 PM | best objective value: 0.6785
11/20/19 @ 11:39 PM | rounding produced 5 integer solutions
11/20/19 @ 11:39 PM | best objective value is 0.6139
11/20/19 @ 11:39 PM | running sequential rounding on 7 solutions
11/20/19 @ 11:39 PM | best objective value: 0.6785
11/20/19 @ 11:39 PM | sequential rounding produced 1 integer solutions
11/20/19 @ 11:39 PM | best objective value: 0.4058
11/20/19 @ 11:39 PM | polishing 6 solutions
11/20/19 @ 11:39 PM | best objective value: 0.4058
11/20/19 @ 11:39 PM | polishing produced 5 integer solutions
11/20/19 @ 1

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 11:39 PM | adding 309 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2806.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.25809112942775264
CPXPARAM_MIP_Tolerances_UpperCutoff              0.28023427690565594


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 11:40 PM | adding 259 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2802.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

11/20/19 @ 11:41 PM | ------------------------------------------------------------
11/20/19 @ 11:41 PM | 213 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.24111422794688847
CPXPARAM_MIP_Tolerances_UpperCutoff              0.26798358679535478


...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:02 AM | adding 304 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3085.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:03 AM | adding 262 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3052.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:04 AM | adding 298 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3054.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:05 AM | adding 252 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3173.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [61]:
np.mean(riskslim_summary['validation_auc']), np.mean(riskslim_summary['test_auc'])

(0.6859939130643332, 0.6782147329035068)

### Single RiskSLIM Model -- Unconstrained

In [63]:
selected_features = ['drug_two_year'] + single_stump_model['features']
sub_train_data = train_stumps[selected_features]
sub_test_data = test_stumps[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
sub_cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [64]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': sub_cols,
    'outcome_name': 'drug_two_year',
    'sample_weights': sample_weights
}

In [65]:
model_info, mip_info, lcpa_info = slim.risk_slim(new_train_data, 
                                                 max_coefficient=5, 
                                                 max_L0_value=5, 
                                                 c0_value=1e-6, 
                                                 max_offset=100, 
                                                 max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
11/21/19 @ 12:06 AM | 213 rows in lookup table
11/21/19 @ 12:06 AM | ------------------------------------------------------------
11/21/19 @ 12:06 AM | runnning initialization procedure
11/21/19 @ 12:06 AM | ------------------------------------------------------------
11/21/19 @ 12:06 AM | CPA produced 2 cuts
11/21/19 @ 12:06 AM | running naive rounding on 7 solutions
11/21/19 @ 12:06 AM | best objective value: 0.6931
11/21/19 @ 12:06 AM | rounding produced 5 integer solutions
11/21/19 @ 12:06 AM | best objective value is 0.6393
11/21/19 @ 12:06 AM | running sequential rounding on 7 solutions
11/21/19 @ 12:06 AM | best objective value: 0.6931
11/21/19 @ 12:06 AM | sequential rounding produced 0 integer solutions
11/21/19 @ 12:06 AM | polishing 5 solutions
11/21/19 @ 12:06 AM | best objective value: 0.6393
11/21/19 @ 12:06 AM | polishing produced 5 integer solutions
11/21/19 @ 12:06 AM | initialization produced 9 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:06 AM | adding 291 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2957.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [66]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [67]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.7218685395212242

In [68]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.6867545511613308

## RiskSLIM -- Constrained

In [70]:
## single stump model
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        #c=0.08, 
                                        c=0.05, 
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

16

In [71]:
selected_features = single_stump_model['features']

In [72]:
### Subset features
if 'sex1' in selected_features:
    selected_features = ['drug_two_year', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['drug_two_year', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0

sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on=['person_id', 'screening_date'])
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [73]:
riskslim_cs_summary = slim.risk_nested_cv_constrain(X=sub_X, 
                                                    Y=sub_Y, 
                                                    indicator = indicator,
                                                    y_label='drug_two_year', 
                                                    max_coef=5,
                                                    max_coef_number=5, 
                                                    max_runtime=1000, 
                                                    c=1e-6, 
                                                    max_offset=100,
                                                    seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/21/19 @ 12:07 AM | 213 rows in lookup table
11/21/19 @ 12:07 AM | ------------------------------------------------------------
11/21/19 @ 12:07 AM | runnning initialization procedure
11/21/19 @ 12:07 AM | ------------------------------------------------------------
11/21/19 @ 12:07 AM | CPA produced 2 cuts
11/21/19 @ 12:07 AM | running naive rounding on 19 solutions
11/21/19 @ 12:07 AM | best objective value: 0.2660
11/21/19 @ 12:07 AM | rounding produced 5 integer solutions
11/21/19 @ 12:07 AM | best objective value is 0.2716
11/21/19 @ 12:07 AM | running sequential rounding on 19 solutions
11/21/19 @ 12:07 AM | best objective value: 0.2660
11/21/19 @ 12:07 AM | sequential rounding produced 6 integer solutions
11/21/19 @ 12:07 AM | best objective value: 0.2709
11/21/19 @ 12:07 AM | polishing 11 solutions
11/21/19 @ 12:07 AM | best objective value: 0.2709
11/21/19 @ 12:07 AM | polishing produced 5 integer solutions
11/21/19 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:07 AM | adding 102 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2709.
Tried aggregator 1 time.
Reduced MIP has 18 rows, 36 columns, and 67 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:07 AM | adding 102 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2725.
Tried aggregator 1 time.
Reduced MIP has 18 rows, 36 columns, and 67 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:07 AM | adding 107 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2572.
Tried aggregator 1 time.
Reduced MIP has 18 rows, 36 columns, and 67 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:07 AM | adding 92 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2583.
Tried aggregator 1 time.
Reduced MIP has 18 rows, 36 columns, and 67 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:08 AM | adding 121 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2933.
Tried aggregator 1 time.
Reduced MIP has 18 rows, 36 columns, and 67 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:08 AM | adding 97 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2906.
Tried aggregator 1 time.
Reduced MIP has 18 rows, 36 columns, and 67 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:08 AM | adding 114 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2937.
Tried aggregator 1 time.
Reduced MIP has 18 rows, 36 columns, and 67 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.02 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:08 AM | adding 100 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2932.
Tried aggregator 1 time.
Reduced MIP has 18 rows, 36 columns, and 67 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [74]:
np.mean(riskslim_cs_summary['validation_auc']), np.mean(riskslim_cs_summary['test_auc'])

(0.686786417088894, 0.7013630222443759)

### Single RiskSLIM Model -- Constrained

In [75]:
selected_features = ['drug_two_year'] + single_stump_model['features']
sub_train_data = train_stumps[selected_features]
sub_test_data = test_stumps[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
sub_cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [76]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': sub_cols,
    'outcome_name': 'drug_two_year',
    'sample_weights': sample_weights
}

In [77]:
model_info, mip_info, lcpa_info = slim.risk_slim_constrain(new_train_data, 
                                                           max_coefficient=5, 
                                                           max_L0_value=5, 
                                                           c0_value=1e-6, 
                                                           max_offset=100, 
                                                           max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
11/21/19 @ 12:08 AM | 213 rows in lookup table
11/21/19 @ 12:08 AM | ------------------------------------------------------------
11/21/19 @ 12:08 AM | runnning initialization procedure
11/21/19 @ 12:08 AM | ------------------------------------------------------------
11/21/19 @ 12:08 AM | CPA produced 2 cuts
11/21/19 @ 12:08 AM | running naive rounding on 28 solutions
11/21/19 @ 12:08 AM | best objective value: 0.2828
11/21/19 @ 12:08 AM | rounding produced 5 integer solutions
11/21/19 @ 12:08 AM | best objective value is 0.2906
11/21/19 @ 12:08 AM | running sequential rounding on 28 solutions
11/21/19 @ 12:08 AM | best objective value: 0.2828
11/21/19 @ 12:08 AM | sequential rounding produced 6 integer solutions
11/21/19 @ 12:08 AM | best objective value: 0.2852
11/21/19 @ 12:08 AM | polishing 11 solutions
11/21/19 @ 12:08 AM | best objective value: 0.2852
11/21/19 @ 12:08 AM | polishing produced 5 integer solutions
11/21/19 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 12:08 AM | adding 117 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2852.
Tried aggregator 1 time.
Reduced MIP has 18 rows, 36 columns, and 67 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [78]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [79]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.7047066097339489

In [80]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.7146892655367232

## Arnold PSA

In [31]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_arnold.csv").sort_values('person_id')
X_arnold = data.loc[:,['arnold_nca', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['drug_two_year'].values

In [32]:
## set up cross validation
cv = KFold(n_splits=5,shuffle=True,random_state=816)
arnold_auc = []
race_auc = []
condition_pn = []
no_condition_pn = []

i = 0
for train, test in cv.split(X_arnold, Y_arnold):
    
    train_x, train_y = X_arnold.iloc[train], Y_arnold[train]
    test_x, test_y = X_arnold.iloc[test], Y_arnold[test]
    holdout_with_attrs = test_x.copy()
    
    ################################
    ## arnold_auc
    arnold_auc.append(roc_auc_score(test_y, test_x['arnold_nca'].values))
    
    ################################
    ## race_auc
    try:
        arnold_race_auc = fairness.fairness_in_auc(df = holdout_with_attrs,
                                                   probs = test_x['arnold_nca'],
                                                   labels = test_y)
        arnold_race_auc_final = arnold_race_auc.assign(fold_num = [i]*arnold_race_auc['Attribute'].count())
        race_auc.append(arnold_race_auc_final)
    except:
        pass
    
    ################################
    ## condition pn
    arnold_condition_pn = fairness.conditional_balance_positive_negative(df = holdout_with_attrs,
                                                                         probs = test_x['arnold_nca'],
                                                                         labels = test_y)
    arnold_condition_pn_final = arnold_condition_pn.assign(fold_num = [i]*arnold_condition_pn['Attribute'].count())
    condition_pn.append(arnold_condition_pn_final)
    
    ################################
    ## no condition pn
    arnold_no_condition_pn = fairness.balance_positive_negative(df = holdout_with_attrs,
                                                                probs = test_x['arnold_nca'],
                                                                labels = test_y)
    arnold_no_condition_pn_final = arnold_no_condition_pn.assign(fold_num = [i]*arnold_no_condition_pn['Attribute'].count())
    no_condition_pn.append(arnold_no_condition_pn_final)
    
    i += 1
    
## race_auc
arnold_race_auc = []
try:
    arnold_race_auc = pd.concat(race_auc, ignore_index=True)
    arnold_race_auc.sort_values(["fold_num", "Attribute"], inplace=True)
    arnold_race_auc = arnold_race_auc.reset_index(drop=True)
except:
    pass

## condition_pn
arnold_condition_pn = pd.concat(condition_pn, ignore_index=True)
arnold_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
arnold_condition_pn = arnold_condition_pn.reset_index(drop=True)

## no_condition_pn
arnold_no_condition_pn = pd.concat(no_condition_pn, ignore_index=True)
arnold_no_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
arnold_no_condition_pn = arnold_no_condition_pn.reset_index(drop=True)

### Single Arnold PSA

In [33]:
test_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_arnold_test.csv")
X = test_data['arnold_nca'].values
Y = test_data['drug_two_year'].values
roc_auc_score(Y, X)

0.6061676082862524

## Compass

In [34]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_arnold.csv").sort_values('person_id')
X_arnold = data.loc[:,['Risk of Recidivism_decile_score', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['drug_two_year'].values

In [35]:
## set up cross validation
cv = KFold(n_splits=5,shuffle=True,random_state=816)
compas_auc = []
race_auc = []
condition_pn = []
no_condition_pn = []

i = 0
for train, test in cv.split(X_arnold, Y_arnold):
    
    train_x, train_y = X_arnold.iloc[train], Y_arnold[train]
    test_x, test_y = X_arnold.iloc[test], Y_arnold[test]
    holdout_with_attrs = test_x.copy()
    
    ################################
    ## arnold_auc
    compas_auc.append(roc_auc_score(test_y, test_x['Risk of Recidivism_decile_score'].values))
    
    ################################
    ## race_auc
    try:
        compas_race_auc = fairness.fairness_in_auc(df = holdout_with_attrs,
                                                   probs = test_x['Risk of Recidivism_decile_score'],
                                                   labels = test_y)
        compas_race_auc_final = compas_race_auc.assign(fold_num = [i]*compas_race_auc['Attribute'].count())
        race_auc.append(compas_race_auc_final)
    except:
        pass
    
    ################################
    ## condition pn
    compas_condition_pn = fairness.conditional_balance_positive_negative(df = holdout_with_attrs,
                                                                         probs = test_x['Risk of Recidivism_decile_score'],
                                                                         labels = test_y)
    compas_condition_pn_final = compas_condition_pn.assign(fold_num = [i]*compas_condition_pn['Attribute'].count())
    condition_pn.append(compas_condition_pn_final)
    
    ################################
    ## no condition pn
    compas_no_condition_pn = fairness.balance_positive_negative(df = holdout_with_attrs,
                                                                probs = test_x['Risk of Recidivism_decile_score'],
                                                                labels = test_y)
    compas_no_condition_pn_final = compas_no_condition_pn.assign(fold_num = [i]*compas_no_condition_pn['Attribute'].count())
    no_condition_pn.append(compas_no_condition_pn_final)
    
    i += 1
    
## race_auc
compas_race_auc = []
try:  
    compas_race_auc = pd.concat(race_auc, ignore_index=True)
    compas_race_auc.sort_values(["fold_num", "Attribute"], inplace=True)
    compas_race_auc = compas_race_auc.reset_index(drop=True)
except:
    pass

## condition_pn
compas_condition_pn = pd.concat(condition_pn, ignore_index=True)
compas_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
compas_condition_pn = compas_condition_pn.reset_index(drop=True)

## no_condition_pn
compas_no_condition_pn = pd.concat(no_condition_pn, ignore_index=True)
compas_no_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
compas_no_condition_pn = compas_no_condition_pn.reset_index(drop=True)

### Results

In [81]:
#### save results
summary_drug2_FL_interpret = {"CART": cart_summary,
                              "EBM": ebm_summary, 
                              'Lasso Stumps': stump_summary, 
                              'RiskSLIM': riskslim_summary, 
                              'RiskSLIM-Constrained': riskslim_cs_summary, 
                              'Arnold PSA': arnold_auc, 
                              'Compas': compas_auc}
%store summary_drug2_FL_interpret

Stored 'summary_drug2_FL_interpret' (dict)


In [82]:
results = [["CART", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])],
           ["EBM", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])], 
           ["Lasso Stumps", np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])],
           ['RiskSLIM', np.mean(riskslim_summary['test_auc'])],
           ['RiskSLIM-Constrained', np.mean(riskslim_cs_summary['test_auc'])],
           ['Arnold PSA', round(np.mean(arnold_auc), 3)], 
           ['Compas', round(np.mean(compas_auc), 3)]]
results

[['CART', 0.6538781478216074, 0.08366322579357326],
 ['EBM', 0.6808643542744772, 0.10732290145669496],
 ['Lasso Stumps', 0.7176427384116442, 0.07189410899365092],
 ['RiskSLIM', 0.6782147329035068],
 ['RiskSLIM-Constrained', 0.7013630222443759],
 ['Arnold PSA', 0.552],
 ['Compas', 0.633]]

In [83]:
auc = [np.mean(cart_summary['holdout_test_auc']), 
       np.mean(ebm_summary['holdout_test_auc']), 
       np.mean(stump_summary['holdout_test_auc']), 
       np.mean(riskslim_summary['test_auc']),
       np.mean(riskslim_cs_summary['test_auc']),]

In [84]:
path = "C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/model results/Interpretable Models/Two Year/"
results = [["Drug", np.str((round(np.mean(cart_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(ebm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(ebm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(stump_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(stump_summary['holdout_test_auc']), 3)) + ")",             
            np.str(round(np.mean(riskslim_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_summary['test_auc']), 3)) + ")", 
            np.str(round(np.mean(riskslim_cs_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_cs_summary['test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3),
            np.str(round(np.mean(arnold_auc), 3)) + " (" + np.str(round(np.std(arnold_auc),3)) + ")", 
            np.str(round(np.mean(compas_auc), 3)) + " (" + np.str(round(np.std(compas_auc),3)) + ")"]]
with open(path + 'Interpretable Models Summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

### Confusion Matrix

In [85]:
cart_confusion = cart_summary['confusion_matrix_stats']
ebm_confusion = ebm_summary['confusion_matrix_stats']
riskslim_confusion = riskslim_summary['confusion_matrix_stats']
riskslim_cs_confusion = riskslim_cs_summary['confusion_matrix_stats']
stumps_confusion = stump_summary['confusion_matrix_stats']

cart_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/two-year/drug/cart_confusion.csv', index=None,header=True)
ebm_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/two-year/drug/ebm_confusion.csv', index=None,header=True)
riskslim_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/two-year/drug/riskslim_confusion.csv', index=None,header=True)
riskslim_cs_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/two-year/drug/riskslim_cs_confusion.csv', index=None,header=True)
stumps_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/two-year/drug/stumps_confusion.csv', index=None,header=True)

### Calibration

In [86]:
cart_calibration = cart_summary['calibration_stats']
ebm_calibration = ebm_summary['calibration_stats']
riskslim_calibration = riskslim_summary['calibration_stats']
riskslim_cs_calibration = riskslim_cs_summary['calibration_stats']
stumps_calibration = stump_summary['calibration_stats']

cart_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/two-year/drug/cart_calibration.csv', index=None,header=True)
ebm_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/two-year/drug/ebm_calibration.csv', index=None,header=True)
riskslim_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/two-year/drug/riskslim_calibration.csv', index=None,header=True)
riskslim_cs_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/two-year/drug/riskslim_cs_calibration.csv', index=None,header=True)
stumps_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/two-year/drug/stumps_calibration.csv', index=None,header=True)

### Race AUC

In [87]:
cart_race_auc = cart_summary['race_auc']
ebm_race_auc = ebm_summary['race_auc']
riskslim_race_auc = riskslim_summary['race_auc']
riskslim_cs_race_auc = riskslim_cs_summary['race_auc']
stumps_race_auc = stump_summary['race_auc']

cart_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/drug/cart_race_auc.csv', index=None,header=True)
ebm_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/drug/ebm_race_auc.csv', index=None,header=True)
riskslim_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/drug/riskslim_race_auc.csv', index=None,header=True)
riskslim_cs_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/drug/riskslim_cs_race_auc.csv', index=None,header=True)
stumps_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/drug/stumps_race_auc.csv', index=None,header=True)
arnold_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/drug/arnold_race_auc.csv', index=None,header=True)
compas_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/drug/compas_race_auc.csv', index=None,header=True)

### Condition PN

In [88]:
cart_condition_pn = cart_summary['condition_pn']
ebm_condition_pn = ebm_summary['condition_pn']
riskslim_condition_pn = riskslim_summary['condition_pn']
riskslim_cs_condition_pn = riskslim_cs_summary['condition_pn']
stumps_condition_pn = stump_summary['condition_pn']

cart_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/drug/cart_condition_pn.csv', index=None,header=True)
ebm_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/drug/ebm_condition_pn.csv', index=None,header=True)
riskslim_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/drug/riskslim_condition_pn.csv', index=None,header=True)
riskslim_cs_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/drug/riskslim_cs_condition_pn.csv', index=None,header=True)
stumps_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/drug/stumps_condition_pn.csv', index=None,header=True)
arnold_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/drug/arnold_condition_pn.csv', index=None,header=True)
compas_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/drug/compas_condition_pn.csv', index=None,header=True)

### No Condition PN

In [89]:
cart_no_condition_pn = cart_summary['no_condition_pn']
ebm_no_condition_pn = ebm_summary['no_condition_pn']
riskslim_no_condition_pn = riskslim_summary['no_condition_pn']
riskslim_cs_no_condition_pn = riskslim_cs_summary['no_condition_pn']
stumps_no_condition_pn = stump_summary['no_condition_pn']

## save results
cart_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/drug/cart_no_condition_pn.csv', index=None,header=True)
ebm_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/drug/ebm_no_condition_pn.csv', index=None,header=True)
riskslim_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/drug/riskslim_no_condition_pn.csv', index=None,header=True)
riskslim_cs_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/drug/riskslim_cs_no_condition_pn.csv', index=None,header=True)
stumps_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/drug/stumps_no_condition_pn.csv', index=None,header=True)
arnold_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/drug/arnold_no_condition_pn.csv', index=None,header=True)
compas_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/drug/compas_no_condition_pn.csv', index=None,header=True)